<a href="https://colab.research.google.com/github/moseskim/bert_nlp/blob/main/section_4/02_fine_tuning_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파인 튜닝을 이용한 감정 분석

파인 튜닝을 활용해 문장의 선호 감정을 판정하는 모델을 훈련합니다.

## 라이브러리 설치

라이브러리인 Transformers 및 nlp를 설치합니다.

In [ ]:
!pip install transformers
!pip install nlp
!pip install dill==0.3.5
!pip install accelerate

## 모델과 Tokenizer 로딩

사전 학습 완료 모델 및 이와 연결된 Tokneizer를 로딩합니다.

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
sc_model.cuda()
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## 데이터셋 로딩

라이브러리인 `nlp`를 사용해 IMDb 데이터셋을 로딩합니다.  
IMDb 데이터셋은 25000개의 영화 리뷰 코멘트에 긍정/부정이라는 선호 감정을 나타내는 라벨을 부여한, 감정 분석용 데이터셋입니다.  
https://www.imdb.com/interfaces/    
로딩한 UMDb 데이터는 토크나이저로 처리하고 형식을 정리합니다.  


In [ ]:
from nlp import load_dataset

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_data, test_data = load_dataset("imdb", split=["train", "test"])
print(train_data["label"][0], train_data["text"][0])  # 호의적인 코멘트
print(train_data["label"][20000], train_data["text"][20000])  # 부정적인 코멘트

train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

test_data = test_data.map(tokenize, batched=True, batch_size=len(train_data))
test_data.set_format("torch", columns=["input_ids", "attention_mask", "label"])

## 평가용 함수

`sklearn.metrics`를 사용해 모델을 평가하기 위한 함수를 정의합니다.


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

## Trainer 설정

`Trainer` 클래스 및 `TrainingArguments` 클래스를 사용해 훈련을 수행하는 `Trainer`를 설정합니다.  
https://huggingface.co/transformers/main_classes/trainer.html   
https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments  



In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    per_gpu_train_batch_size = 8,
    warmup_steps = 500,  # 학습 계수는 0부터 이 단계수로 상승
    weight_decay = 0.01,  # 가중치 감쇠율
    # evaluate_during_training = True,  # 이 부분은 버전에 따라서는 필요하지 않음
    logging_dir = "./logs",
)

trainer = Trainer(
    model = sc_model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = test_data
)

## 모델 훈련

설정에 기반해 모델을 훈련합니다.

In [ ]:
trainer.train()

## 모델 평가

Trainer의 `evaluate()` 메서드를 사용해 모델을 평가합니다.  

In [ ]:
trainer.evaluate()

## TensorBoard를 사용한 결과 표시

TensorBoard를 사용해 `logs` 폴더에 저장된 학습 과정을 표시합니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs